In [1]:
from SpacetimeVar import *

In [2]:
DiffOrder = 4
nghostcells = 4
KOsigma = 0.1

In [3]:
dim = 3
path = "../Source/HamCom/"

In [4]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")

In [5]:
RHS_Header = """#ifndef ET_INTEGRATION_RHS_K_H 
#define ET_INTEGRATION_RHS_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_rhs(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& rhs_fab, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, 
        const amrex::GeometryData& geom) noexcept 
{
        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2]; 
"""

fileRHS.write(RHS_Header)

705

In [6]:
stVar.decl = []
u = stVar('u', declare = True)

In [7]:
fileRHS.write(u.AMReXDeclare())

52

In [8]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write("names = {")
for itr in range(len(stVar.decl)-1):
    fileVARS.write("\""+stVar.decl[itr]+"\", ")
fileVARS.write("\""+stVar.decl[len(stVar.decl)-1]+"\"")
fileVARS.write("};")
fileVARS.close()

In [9]:
fileRHS.write(DstVar(u, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(u, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(u, 'KO', orderD = DiffOrder).AMReXDeclare())

835

In [10]:
x = stVarRank1('x')
r = stVar('r')
r.var = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)

c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.var = np.array([1.168642873,0,0])
c2.var = np.array([-1.168642873,0,0])

M1 = stVar('M1')
M2 = stVar('M2')

M1.var = 0.453
M2.var = 0.453

P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.var = np.array([0,0.3331917498,0])
P2.var = np.array([0,-0.3331917498,0])

xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.var[i] = x.symb[i]-c1.symb[i]
    xc2.var[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.var = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.var = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

eps1 = stVar('eps1')
eps2 = stVar('eps2')

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.var[i] = xc1.symb[i]/rc1.symb
    n2.var[i] = xc2.symb[i]/rc2.symb

In [11]:
fileRHS.write(r.AMReXReal())
fileRHS.write(c1.AMReXReal())
fileRHS.write(c2.AMReXReal())
fileRHS.write(M1.AMReXReal())
fileRHS.write(M2.AMReXReal())
fileRHS.write(P1.AMReXReal())
fileRHS.write(P2.AMReXReal())
fileRHS.write(xc1.AMReXReal())
fileRHS.write(xc2.AMReXReal())
fileRHS.write(rc1.AMReXReal())
fileRHS.write(rc2.AMReXReal())
fileRHS.write(n1.AMReXReal())
fileRHS.write(n2.AMReXReal())

108

In [12]:
Atilde1UU = stVarRank2('Atilde1UU')
Atilde2UU = stVarRank2('Atilde2UU')
for i in range(3):
    for j in range(3):
        Atilde1UU.var[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Atilde2UU.var[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Atilde1UU.var[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Atilde2UU.var[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]

In [13]:
AtildeUU = stVarRank2('AtildeUU')
for i in range(3):
    for j in range(3):
        AtildeUU.var[i][j] += Atilde1UU.symb[i][j]+Atilde2UU.symb[i][j]

In [14]:
fileRHS.write(Atilde1UU.AMReXReal())
fileRHS.write(Atilde2UU.AMReXReal())
fileRHS.write(AtildeUU.AMReXReal())

540

In [15]:
alpha = stVar('alpha')

alpha.var = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [16]:
fileRHS.write(alpha.AMReXReal())

71

In [17]:
beta = stVar('beta')

for i in range(3):
    for j in range(3):
        beta.var += 1/8*(alpha.symb**7)*AtildeUU.symb[i][j]*AtildeUU.symb[i][j]

In [18]:
fileRHS.write(beta.AMReXReal())

489

In [19]:
RHS_u = stVar('u')

RHS_u.var = 0
for i in range(3):
    RHS_u.var += Dsymb(u.symb,str(i)+str(i))
    
RHS_u.var += beta.symb*(alpha.symb*(1+u.symb)+1)**(-7)

RHS_u.var += Dsymb(u.symb,'KO')

In [20]:
fileRHS.write(RHS_u.AMReXRHS())

103

In [21]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

In [22]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")

In [23]:
DIAG_Header = """#ifndef ET_INTEGRATION_DIAG_K_H 
#define ET_INTEGRATION_DIAG_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_diagnostics(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& diag, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx,
        const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2];
"""

fileDIAG.write(DIAG_Header)

711

In [24]:
fileDIAG.write(u.AMReXDeclare())

52

In [25]:
x = stVarRank1('x')
r = stVar('r')
r.var = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)

c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.var = np.array([1.168642873,0,0])
c2.var = np.array([-1.168642873,0,0])

M1 = stVar('M1')
M2 = stVar('M2')

M1.var = 0.453
M2.var = 0.453

P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.var = np.array([0,0.3331917498,0])
P2.var = np.array([0,-0.3331917498,0])

xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.var[i] = x.symb[i]-c1.symb[i]
    xc2.var[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.var = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.var = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

eps1 = stVar('eps1')
eps2 = stVar('eps2')

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.var[i] = xc1.symb[i]/rc1.symb
    n2.var[i] = xc2.symb[i]/rc2.symb

In [26]:
fileDIAG.write(r.AMReXReal())
fileDIAG.write(c1.AMReXReal())
fileDIAG.write(c2.AMReXReal())
fileDIAG.write(M1.AMReXReal())
fileDIAG.write(M2.AMReXReal())
fileDIAG.write(P1.AMReXReal())
fileDIAG.write(P2.AMReXReal())
fileDIAG.write(xc1.AMReXReal())
fileDIAG.write(xc2.AMReXReal())
fileDIAG.write(rc1.AMReXReal())
fileDIAG.write(rc2.AMReXReal())
fileDIAG.write(n1.AMReXReal())
fileDIAG.write(n2.AMReXReal())

108

In [27]:
Abar1UU = stVarRank2('Abar1UU')
Abar2UU = stVarRank2('Abar2UU')
for i in range(3):
    for j in range(3):
        Abar1UU.var[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Abar2UU.var[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Abar1UU.var[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Abar2UU.var[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]

In [28]:
AbarUU = stVarRank2('AbarUU')
for i in range(3):
    for j in range(3):
        AbarUU.var[i][j] += Abar1UU.symb[i][j]+Abar2UU.symb[i][j]

In [29]:
fileDIAG.write(Abar1UU.AMReXReal())
fileDIAG.write(Abar2UU.AMReXReal())
fileDIAG.write(AbarUU.AMReXReal())

486

In [30]:
alphafunc = stVar('alphafunc')

alphafunc.var = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [31]:
fileDIAG.write(alphafunc.AMReXReal())

71

In [32]:
psi = stVar('psi')

psi.var = 1+alphafunc.symb**(-1)+u.symb

In [33]:
fileDIAG.write(psi.AMReXReal())

45

In [34]:
stVar.declDiag = []  #We want a list of declared variables that we will cycle through when we convert our RHS equations.

chi = stVar('chi',declareDiag = True) #Scalar variables like chi can be declared with stVar.
gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01', declareDiag = True)  #gamtildeLL is rank 2, so it is declared with stVarRank2
Khat = stVar('Khat',declareDiag = True)
AtildeLL = stVarRank2('AtildeLL', sym = 'sym01',declareDiag = True)
theta = stVar('theta',declareDiag = True)
GamtildeU = stVarRank1('GamtildeU',declareDiag = True)

alpha = stVar('alpha',declareDiag = True)
betaU = stVarRank1('betaU',declareDiag = True)

In [35]:
fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write("names = {")
for itr in range(len(stVar.declDiag)-1):
    fileDIAGVARS.write("\""+stVar.declDiag[itr]+"\", ")
fileDIAGVARS.write("\""+stVar.declDiag[len(stVar.declDiag)-1]+"\"")
fileDIAGVARS.write("};")
fileDIAGVARS.close()

In [36]:
chi.var = psi.symb**(-4)

In [37]:
fileDIAG.write(chi.AMReXDiag())

55

In [38]:
gamtildeLL.var = [[1,0,0],[0,1,0],[0,0,1]]

In [39]:
fileDIAG.write(gamtildeLL.AMReXDiag())

288

In [40]:
Khat.var = 0

In [41]:
fileDIAG.write(Khat.AMReXDiag())

40

In [42]:
AtildeLL.var = (psi.symb**(-6))*AbarUU.symb

In [43]:
fileDIAG.write(AtildeLL.AMReXDiag())

420

In [44]:
theta.var = 0

In [45]:
fileDIAG.write(theta.AMReXDiag())

41

In [46]:
GamtildeU.var = [0,0,0]

In [47]:
fileDIAG.write(GamtildeU.AMReXDiag())

138

In [48]:
alpha.var = psi.symb**(-2)

In [49]:
fileDIAG.write(alpha.AMReXDiag())

57

In [50]:
betaU.var = [0,0,0]

In [51]:
fileDIAG.write(betaU.AMReXDiag())

126

In [52]:
fileDIAG.write("}\n")
fileDIAG.write("#endif")
fileDIAG.close()

In [53]:
fileSETUP = open(path+"ET_Integration_Setup.H", "w+")

In [54]:
fileSETUP.write("#ifndef ET_INTEGRATION_SETUP_K_H \n")
fileSETUP.write("#define ET_INTEGRATION_SETUP_K_H \n\n")

fileSETUP.write("#include <AMReX_REAL.H> \n")
fileSETUP.write("#include <AMReX_Array4.H> \n\n")
    
fileSETUP.write("namespace Idx { \n")
fileSETUP.write("         enum ETIndexes {")
    
Idx_string = ""
for itr in stVar.decl:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("namespace Diag { \n")
fileSETUP.write("         enum DiagnosticIndexes {")
    
Idx_string = ""
for itr in stVar.declDiag:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("#define NUM_GHOST_CELLS "+str(nghostcells)+"\n\n")
fileSETUP.write("#endif")

fileSETUP.close()

In [55]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")

In [56]:
PostUpdate_Header = """#ifndef ET_INTEGRATION_POST_UPDATE_K_H 
#define ET_INTEGRATION_POST_UPDATE_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_post_update(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        const amrex::GeometryData& geom) noexcept 
{
"""

filePOST.write(PostUpdate_Header)

348

In [57]:
filePOST.write("}\n")
filePOST.write("#endif")
filePOST.close()

In [58]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")

In [59]:
Init_Header = """#ifndef ET_INTEGRATION_INIT_K_H 
#define ET_INTEGRATION_INIT_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_init(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        amrex::Real time, const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2]; 
"""

fileINIT.write(Init_Header)

605

In [60]:
uInit = stVar('u')

In [61]:
uInit.var = 0

In [62]:
fileINIT.write(uInit.AMReXInit())

41

In [63]:
fileINIT.write("}\n")
fileINIT.write("#endif")
fileINIT.close()